In [1]:
!pip install qdrant-client fastembed openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 15.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.1
    Uninstalling huggingface-hub-0.22.1:
      Successfully uninstalled huggingface-hub-0.22.1


In [2]:
!docker run -p "6333:6333" -p "6334:6334" --name "rag-openai-qdrant" --rm -d qdrant/qdrant:latest

Unable to find image 'qdrant/qdrant:latest' locally
latest: Pulling from qdrant/qdrant

764b1f6d: Pulling fs layer 
1bbfc367: Pulling fs layer 
1aa84de5: Pulling fs layer 
87a9373b: Pulling fs layer 
144233c8: Pulling fs layer 
3d86a88d: Pulling fs layer 
9e054cc5: Pulling fs layer 
b700ef54: Pull complete   32B/32B87MBBDigest: sha256:74d60f1d452520c8af3ac41a1e7e671ca7fb3b7569b8728af523136a2019a878
Status: Downloaded newer image for qdrant/qdrant:latest
6835aa02f32ce91c32a6503f06dfbc3eb0d2754f2be1e64b47dfbb6ab58023b8


In [3]:
import qdrant_client

client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='test_collection'), CollectionDescription(name='news_embeddings'), CollectionDescription(name='better_news'), CollectionDescription(name='news_collection')])

In [5]:
client.add(
    collection_name="knowledge-base",
    documents=[
        "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
        "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
        "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
        "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
        "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
        "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
        "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
        "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
    ]
)

100%|█████████████████████████████████████| 77.7M/77.7M [00:05<00:00, 14.2MiB/s]


['00dec2728ab74180b94dd5136df28a6f',
 '59e086d5674049d3bcbe2116b8c6a81e',
 'e8fd07c001724476ba7f75e542b061d7',
 '8fa5a12839044de9b9714f0ae8da35b6',
 '7cebf4344ee44333a49d25fcac735169',
 '29a4698ffc734093bed798a869f99777',
 '745d3177ce0040f2a8462e713aee96b8',
 'a87a8dfc7ee24b1bbf2e276e8d488a51']

In [6]:
prompt = """
What tools should I need to use to build a web service using vector embeddings for search?
"""

In [8]:
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
from openai import OpenAI

openai_client = OpenAI(
    api_key=OPENAI_KEY
)

completion = openai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)
print(completion.choices[0].message.content)

To build a web service using vector embeddings for search, you will need the following tools:

1. Programming language: Choose a programming language that supports machine learning and deep learning frameworks such as Python, Java, or C++. Python is commonly used for building web services with machine learning capabilities.

2. Machine learning framework: Utilize machine learning frameworks such as TensorFlow, PyTorch, or Scikit-learn for creating and training the vector embeddings model.

3. Web framework: Select a web framework such as Flask or Django for building the web service. These frameworks will help you handle requests, manage routes, and serve the model's predictions.

4. Deployment platform: Choose a platform for deploying your web service, such as Docker or a cloud platform like AWS, Azure, or Google Cloud Platform.

5. Database: Set up a database to store and retrieve the vector embeddings and associated metadata for efficient search operations. Options include SQL databa

In [23]:
results = client.query(
    collection_name="knowledge-base",
    query_text=prompt,
    limit=3,
)

results

[QueryResponse(id='00dec272-8ab7-4180-b94d-d5136df28a6f', embedding=None, sparse_embedding=None, metadata={'document': 'Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!'}, document='Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!', score=0.829069972038269),
 QueryResponse(id='29a4698f-fc73-4093-bed7-98a869f99777', embedding=None, sparse_embedding=None, metadata={'document': 'FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Pyth

In [24]:
context = "\n".join(r.document for r in results)
context

'Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!\nFastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.\nPyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.'

In [25]:
metaprompt = f"""
You are a software architect. 
Answer the following question using the provided context. 
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: {prompt.strip()}

Context: 
{context.strip()}

Answer:
"""

# Look at the full metaprompt
print(metaprompt)


You are a software architect. 
Answer the following question using the provided context. 
If you can't find the answer, do not pretend you know it, but answer "I don't know".

Question: What tools should I need to use to build a web service using vector embeddings for search?

Context: 
Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.
PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.

Answer:



In [35]:
completion_new = openai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[
        {"role": "user", "content": metaprompt}
    ]
)

completion_new

ChatCompletion(id='chatcmpl-99d53yEz7xEoAWvHixSHbvJYtWdot', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You would need Qdrant for the vector database and similarity search engine, FastAPI for building the API service, and PyTorch for working with embeddings or neural network encoders.', role='assistant', function_call=None, tool_calls=None))], created=1712082205, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=37, prompt_tokens=188, total_tokens=225))

In [36]:
print(completion_new.choices[0].message.content)

You would need Qdrant for the vector database and similarity search engine, FastAPI for building the API service, and PyTorch for working with embeddings or neural network encoders.


In [37]:
def rag(question: str, n_points: int = 3) -> str:
    results = client.query(
        collection_name="knowledge-base",
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect. 
    Answer the following question using the provided context. 
    If you can't find the answer, do not pretend you know it, but answer "I don't know".
    
    Question: {question.strip()}
    
    Context: 
    {context.strip()}
    
    Answer:
    """

    completion_new = openai_client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[
            {"role": "user", "content": metaprompt}
        ],
        timeout=10.0,
    )
    
    return completion_new.choices[0].message.content

In [38]:
rag("What can the stack for a web api look like?")

'The stack for a web API using FastAPI, Docker, and NGINX could look like this:\n1. FastAPI for building the API endpoints and handling HTTP requests/responses.\n2. Docker to containerize the FastAPI application for easy deployment and management.\n3. NGINX as a reverse proxy server to handle incoming client requests, load balancing, and potentially serving static content.\nOverall, this stack would provide a high-performance, scalable, and reliable environment for hosting a web API.'

In [39]:
rag("Where is the nearest grocery store?")

"I don't know."

In [40]:
!docker kill rag-openai-qdrant
!docker rm rag-openai-qdrant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


rag-openai-qdrant
Error response from daemon: No such container: rag-openai-qdrant


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
